In [33]:
import numpy as np
import pandas as pd
import csv
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## Indetify missing geo coord import ones and create the list of addresses for re-trial

In [3]:
df = pd.read_csv('data/first_round_geo_imported.csv')

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,20170928,103131,2017-09-28,0,0,5,0,0,6,5
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,20161206,103157,2016-12-06,0,6,0,0,5,5,3
2,2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,37.754343,...,20160503,103120,2016-05-03,4,0,2,0,3,3,2
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
4,4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,37.795305,...,20160329,103113,2016-03-29,0,11,0,0,0,6,6


In [7]:
mask = df['business_latitude'].isnull().values

In [9]:
df[mask].head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
11,11,11,11,82880,Don Pablo,Off The Grid,San Francisco,CA,NaN,NaN,...,20160309,103148,2016-03-09,0,0,0,3,0,6,2


In [11]:
len(df[mask])

742

In [13]:
# 742 businesses where geo coords were not transferred
df_742 = df[mask]

In [17]:
address_no_geocoord = df_742['business_address']

In [18]:
address_nogeo_lst = address_no_geocoord.values

In [28]:
num_vars = ['Off the Grid', 'Off The Grid', 'OFF The Grid', 'OFF THE GRID', 'Approved private locations', 'Approved Private Locations','Approved Locations']
L_nogeo = []
for address in address_nogeo_lst:
    #if address != 'Off the Grid' and address != 'Off The Grid':
    if ~(address in num_vars):
        L_nogeo.append(address)

In [30]:
L_nogeo

['498 09th St Ste. C',
 'Off The Grid',
 'Pier 45  Shed B Space 1A',
 '2301 Market St',
 '2740 Mission St',
 '2348 Mission Street',
 '2800 California St #102',
 '100 Van Ness Ave #A',
 '717 Battery St Basement',
 '5937 Geary Blvd',
 '200 Columbus St',
 '977 A North Point St',
 '135 04th St',
 '1548 Ocean Ave',
 '2331 Chestnut St',
 '1125 Ocean Ave 102',
 '123 02nd St',
 '77 Battery St Suite 100',
 '2404 California St',
 '5601 Geary Blvd',
 '4517 03rd St',
 '401 13th St',
 '71 Stevenson St #100',
 '2794 California St',
 '1919 Ocean Ave',
 '3985 17th St',
 '295 B Orizaba Ave',
 '855 Terry Francois Blvd',
 '3336 Sacramento Ave',
 'Off The Grid',
 '3348 Steiner St',
 '2893 Mission St',
 'Pier 41',
 'Pier 39  B-4',
 '1019 Market St',
 '700 South Van Ness Ave',
 '1 Pagoda Pl',
 '2848 23rd St',
 '817 TERRY FRANCOIS Blvd',
 '2470 San Bruno Ave',
 '1350 07th Ave',
 '561 Columbus Ave',
 '31 Cambon Dr',
 '5200 Geary Blvd',
 '12 Tillman St',
 '5251 3rd St.',
 '5800 3rd St  C1001',
 '1524 \\ Ave',


In [34]:
## Let's run geopy

In [35]:
geolocator = Nominatim(timeout = 10)

In [48]:
location = geolocator.geocode("2740 Mission St SF CA")
print(location.address, location.longitude, location.latitude)

Mission Street, St. Mary's Park, SF, California, 94134, United States of America -122.4291725 37.7308875


In [38]:
# add SF CA to addresses
L_nogeo_SFCA = []
for address in L_nogeo:
    L_nogeo_SFCA.append(address + ' SF CA')

In [39]:
def do_geocode2(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        time.sleep(1)
        return do_geocode(address)

In [40]:
write_file = "data/geo_coords_sf_742.csv"
with open(write_file, "w") as output:
    for address in L_nogeo_SFCA[0:742]:  
        location = do_geocode2(address)
        if location != None:
            output.write( location.address + ',' + str(location.longitude) + ',' + str(location.latitude) +  '\n')

In [47]:
with open('data/geo_coords_sf_742.csv', 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        counter += 1
        print(row)
print(counter)

['Off the Grid', ' Marina Boulevard', ' Marina District', ' SF', ' California', ' 94114', ' United States of America', '-122.4314265', '37.806651']
['Market Street', ' Financial District', ' SF', ' California', ' 94114', ' United States of America', '-122.3987948', '37.7913412']
['Mission Street', " St. Mary's Park", ' SF', ' California', ' 94134', ' United States of America', '-122.4291725', '37.7308875']
['Mission Street', " St. Mary's Park", ' SF', ' California', ' 94134', ' United States of America', '-122.4291725', '37.7308875']
['Geary Boulevard', ' Western Addition', ' SF', ' California', ' 94114', ' United States of America', '-122.4255764', '37.7851991']
['Columbus Street', ' North Beach', ' SF', ' California', ' 94114', ' United States of America', '-122.4088136', '37.7994755']
['977', ' North Point Street', ' Russian Hill', ' SF', ' California', ' 94109', ' United States of America', '-122.423183959184', '37.8052633877551']
['Ocean Avenue', ' Ingleside', ' SF', ' California'

## Create a file with business name, longitudinal, and latitude

In [42]:
write_file = "data/address_geo_742.csv"
with open(write_file, "w") as output:
    with open('data/geo_coords_sf_742.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            n = len(row)   # number of strings in a row. '2449', ' 23rd Street', ' Potrero Terrace', ...
            # below is for a line
            for i in range(2):  # iterate each string in a row. i is the position of a string within a row.
                row[i] = row[i].strip()  # strip off the white space in front
                if row[i].isdigit():  # this will enable to skip the business name in front
                    address = row[i] + row[i+1]
                    
                    if row[i+2].strip() == 'SF':
                        longitude = float(row[i+6])
                        latitude = float(row[i+7])
                    elif row[i+3].strip() == 'SF':
                        longitude = float(row[i+7])
                        latitude = float(row[i+8])
                       
                    output.write(address + ',' + str(longitude) + ',' + str(latitude) +  '\n')
                    break 

In [46]:
# show the rows of geo_coords_sf.csv
with open('data/geo_coords_sf_742.csv', 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        print(row)
        counter += 1
        if counter > 10: break

['Off the Grid', ' Marina Boulevard', ' Marina District', ' SF', ' California', ' 94114', ' United States of America', '-122.4314265', '37.806651']
['Market Street', ' Financial District', ' SF', ' California', ' 94114', ' United States of America', '-122.3987948', '37.7913412']
['Mission Street', " St. Mary's Park", ' SF', ' California', ' 94134', ' United States of America', '-122.4291725', '37.7308875']
['Mission Street', " St. Mary's Park", ' SF', ' California', ' 94134', ' United States of America', '-122.4291725', '37.7308875']
['Geary Boulevard', ' Western Addition', ' SF', ' California', ' 94114', ' United States of America', '-122.4255764', '37.7851991']
['Columbus Street', ' North Beach', ' SF', ' California', ' 94114', ' United States of America', '-122.4088136', '37.7994755']
['977', ' North Point Street', ' Russian Hill', ' SF', ' California', ' 94109', ' United States of America', '-122.423183959184', '37.8052633877551']
['Ocean Avenue', ' Ingleside', ' SF', ' California'

In [7]:
# count the rows of address_geo.csv
with open('data/address_geo.csv', 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for row in reader:
        counter += 1
        #print(row)
print(counter)

1474


## list of geo coords

In [8]:
df = pd.read_csv('data/ready4model_v2.csv')

In [9]:
mask = df['business_latitude'].isnull().values
mask.sum()

2231

In [10]:
df.head(20)

,Unnamed: 0,Unnamed: 0.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,...,20170928,103131,2017-09-28,0,0,5,0,0,6,5
1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,...,20161206,103157,2016-12-06,0,6,0,0,5,5,3
2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,...,20160503,103120,2016-05-03,4,0,2,0,3,3,2
3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,...,20160329,103113,2016-03-29,0,11,0,0,0,6,6
5,5,5,82123,Cafe New Honolulu,888 Stockton St,San Francisco,CA,94108,NaN,NaN,...,20171207,103131,2017-12-07,0,3,0,0,0,8,8
6,6,6,1146,Aux Delices Vietnam,2327 Polk St,San Francisco,CA,94109,37.798274,-122.422264,...,20180223,103103,2018-02-23,6,0,0,0,0,8,0
7,7,7,3838,CAFE PICARO,3120 16th St,San Francisco,CA,94103,37.764908,-122.422442,...,20180222,103142,2018-02-22,7,0,0,0,0,1,2
8,8,8,38929,Jelly Donut,3198 24th St,San Francisco,CA,94110,37.752439,-122.416250,...,20160114,103139,2016-01-14,0,0,7,0,3,4,6
9,9,9,81161,Limon Peruvian Rotisserie,1001 South Van Ness Ave,San Francisco,CA,94110,NaN,NaN,...,20160325,103149,2016-03-25,1,0,3,2,4,0,3


In [11]:
len(df)

5229

In [12]:
with open('data/address_geo.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        address = row[0]
        longitude = row[1]
        latitude = row[2]
        n = len(df)
        for idx in range(n):
            df_address = df.loc[idx,'business_address'].lower()
            if df_address[:10] == address.lower()[:10]:
                df.loc[idx,'business_longitude'] = longitude
                df.loc[idx,'business_latitude'] = latitude

In [18]:
df.head(20)

,Unnamed: 0,Unnamed: 0.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,...,short_inspect_date,short_violation_id,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36
0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.7898,-122.42,...,20170928,103131,2017-09-28,0,0,5,0,0,6,5
1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.7592,-122.419,...,20161206,103157,2016-12-06,0,6,0,0,5,5,3
2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,37.7543434081633,-122.403988857143,...,20160503,103120,2016-05-03,4,0,2,0,3,3,2
3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,...,20171207,103105,2017-12-07,0,5,0,0,6,0,0
4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,37.795304999856,-122.406613368069,...,20160329,103113,2016-03-29,0,11,0,0,0,6,6
5,5,5,82123,Cafe New Honolulu,888 Stockton St,San Francisco,CA,94108,37.7938858979592,-122.407763040816,...,20171207,103131,2017-12-07,0,3,0,0,0,8,8
6,6,6,1146,Aux Delices Vietnam,2327 Polk St,San Francisco,CA,94109,37.7983,-122.422,...,20180223,103103,2018-02-23,6,0,0,0,0,8,0
7,7,7,3838,CAFE PICARO,3120 16th St,San Francisco,CA,94103,37.7649,-122.422,...,20180222,103142,2018-02-22,7,0,0,0,0,1,2
8,8,8,38929,Jelly Donut,3198 24th St,San Francisco,CA,94110,37.7524,-122.416,...,20160114,103139,2016-01-14,0,0,7,0,3,4,6
9,9,9,81161,Limon Peruvian Rotisserie,1001 South Van Ness Ave,San Francisco,CA,94110,37.7570488,-122.4165149,...,20160325,103149,2016-03-25,1,0,3,2,4,0,3


In [14]:
# count of no geo coords
mask = df['business_latitude'].isnull().values

In [15]:
mask.sum()

742

In [16]:
df.count()

Unnamed: 0               5229
Unnamed: 0.1             5229
business_id              5229
business_name            5229
business_address         5229
business_city            5229
business_state           5229
business_postal_code     5070
business_latitude        4487
business_longitude       4487
business_location        2998
business_phone_number    1582
inspection_id            5229
inspection_date          5229
inspection_score         4808
inspection_type          5229
violation_id             5229
violation_description    5229
risk_category            5229
short_inspect_date       5229
short_violation_id       5229
inspect_date             5229
p1_3                     5229
p4_6                     5229
p7_9                     5229
p10_12                   5229
p13_18                   5229
p19_24                   5229
p25_36                   5229
dtype: int64

In [17]:
df.to_csv('data/first_round_geo_imported.csv')